In [1]:
# selecting the stocks we want to use in portfolio optimization
# we will add the symbols of the stock in a list 

symbols = ['AMZN', 'TSLA', 'NVDA', 'AAPL', 'META', 'GOOGL', 'AMD', 'MSFT', 'PYPL', 'INTC']

In [2]:
# now we will fetch the data from alpha vantage api
# we will use symbol list to get data of multiple stocks
import requests
import pandas as pd
import time

def data_to_mongo(df):
    sym = []
    for j in df['symbol'].unique():
        sym.append(j)
    
    for com in sym:
        data = df[df['symbol'] == com]
        data_dict = {
            'index' :[], 
            'symbol' :[],
            'open_' :[],
            'high' :[],
            'low' :[],
            'close' :[],
            'volume' :[]
        }
        list_index = []
        for i in data['index']:
            list_index.append(i)

        list_symbol = []
        for i in data['symbol']:
            list_symbol.append(i)

        list_open_ = []
        for i in data['open_']:
            list_open_.append(i)

        list_high = []
        for i in data['high']:
            list_high.append(i)

        list_low = []
        for i in data['low']:
            list_low.append(i)

        list_close = []
        for i in data['close']:
            list_close.append(i)

        list_volume = []
        for i in data['volume']:
            list_volume.append(i)
        
        data_dict['index'].append(list_index)
        data_dict['symbol'].append(list_symbol)
        data_dict['open_'].append(list_open_)
        data_dict['high'].append(list_high)
        data_dict['low'].append(list_low)
        data_dict['close'].append(list_close)
        data_dict['volume'].append(list_volume)
    
        import pymongo
        client = pymongo.MongoClient("mongodb://localhost:27017")

        database = client["stock_data"]
        collection = database[com]
        collection.insert_one(data_dict)
        
        
df = pd.DataFrame()

for stock in symbols:
    url = "https://alpha-vantage.p.rapidapi.com/query"

    querystring = {"function":"TIME_SERIES_DAILY","symbol":stock,"outputsize":"full","datatype":"json"}

    headers = {
        "X-RapidAPI-Key": "a144766ef7msh26ad66bb50c978cp11719fjsnc707b491b156",
        "X-RapidAPI-Host": "alpha-vantage.p.rapidapi.com"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    data = pd.DataFrame(response.json())
    
    data['symbol'] = data['Meta Data']['2. Symbol']
    data = data.reset_index(drop=False)
    data.drop([0,1,2,3,4], axis=0, inplace=True)
    data.drop('Meta Data', axis=1, inplace=True)
    
    data = data.reset_index(drop=True)
    
    open_ = []
    high = []
    low = []
    close = []
    volume = []
    for i in range(len(data['Time Series (Daily)'])):
        open_.append(data['Time Series (Daily)'][i]['1. open'])
        high.append(data['Time Series (Daily)'][i]['2. high'])
        low.append(data['Time Series (Daily)'][i]['3. low'])
        close.append(data['Time Series (Daily)'][i]['4. close'])
        volume.append(data['Time Series (Daily)'][i]['5. volume'])
    
    data['open_'] = open_
    data['high'] = high
    data['low'] = low
    data['close'] = close
    data['volume'] = volume
    
    data.drop('Time Series (Daily)', axis=1, inplace=True)
    
    data.rename(columns = {"index": "date"})
    
    df = pd.concat([df, data], axis = 0, ignore_index = True)
    
    time.sleep(10)
    
data_to_mongo(df)

In [3]:
df

,index,symbol,open_,high,low,close,volume
0,2023-03-21,AMZN,98.1400,100.8450,98.0000,100.6100,58597275
1,2023-03-20,AMZN,98.4100,98.4800,95.7000,97.7100,62388911
2,2023-03-17,AMZN,99.7900,100.6600,97.4550,98.9500,87300242
3,2023-03-16,AMZN,95.7500,100.9900,95.6100,100.0400,84558934
4,2023-03-15,AMZN,93.2200,96.6700,93.0700,96.2000,70731792
...,...,...,...,...,...,...,...
47841,1999-11-05,INTC,82.3800,83.8800,81.8100,82.3800,19837200
47842,1999-11-04,INTC,80.4400,81.7500,80.1900,81.5600,21843500
47843,1999-11-03,INTC,78.3800,80.6900,77.7500,79.7500,25240100
47844,1999-11-02,INTC,77.0600,78.1900,76.5600,77.5000,20338200
